In [ ]:
%pip install pandas openpyxl
import pandas as pd

df_24 = pd.read_excel("orcamentos/orcamento_2024_V2.xlsx", engine="openpyxl")
df_25 = pd.read_excel("orcamentos/Orcamento_2025_V2.xlsx", engine="openpyxl")
(df_24.shape, df_25.shape)

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


((386, 18), (1224, 18))

In [ ]:
df_25.rename(columns={'Projeto': 'Codigo_2025'}, inplace=True)
df_24.columns = [col.split('_2024')[0].replace('ç', 'c') for col in df_24.columns]
df_25.columns = [col.split('_2025')[0].replace('ç', 'c') for col in df_25.columns]


df_24['Ano_Cotacao'] = 2024
df_25['Ano_Cotacao'] = 2025



In [ ]:
df_24.rename(columns={'Software_Desenvolv_do_Internamente': 'Software_Desenvolvido_Internamente',
                      'Produtos_Desenvolv_dos_Internamente': 'Produtos_Desenvolvidos_Internamente',
                      'Centro_de_Custo': 'Centro_Custo'},inplace=True)

In [ ]:
set(df_24.columns) - set(df_25.columns)

set()

In [ ]:
# Substituir pontos por nada e vírgulas por pontos
#df_24['Valor'] = df_24['Valor'].astype(str).str.replace('.', '')
#df_24['Valor'] = df_24['Valor'].astype(str).str.replace(',', '.')


In [ ]:
# Tentar converter a coluna para float e lidar com erros
df_24['Valor'] = pd.to_numeric(df_24['Valor'], errors='coerce')
df_25['Valor'] = pd.to_numeric(df_25['Valor'], errors='coerce')

In [ ]:
df_merged = pd.concat([df_25, df_24])
df_merged.reset_index(drop=True, inplace=True)

df_merged['Projeto_padronizado'] = df_merged['Projeto'].str.lower().str.replace('pardini', ' ').str.replace('fleury', ' ').str.replace('/', ' ').str.strip()
df_merged['Descricao_padronizado'] = df_merged['Descricao'].str.lower().str.replace('pardini', ' ').str.replace('fleury', ' ').str.replace('/', ' ').str.strip()

df_merged.shape

(1610, 21)

In [ ]:
#df_merged.to_excel('orcamento_2024_2025.xlsx')

In [ ]:
!pip install sentence_transformers

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
from sentence_transformers import SentenceTransformer

# Carregar o modelo pré-treinado SBERT
#model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = SentenceTransformer('BAAI/bge-m3')

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

df_2024 = df_merged[df_merged['Ano_Cotacao'] == 2024].reset_index(drop=True)
df_2025 = df_merged[df_merged['Ano_Cotacao'] == 2025].reset_index(drop=True)

# Supondo que `texts` seja a lista combinada de Projeto e Descrição
texts_24 = df_2024['Projeto_padronizado'].astype(str) + ' ' + df_2024['Descricao_padronizado'].astype(str)
texts_25 = df_2025['Projeto_padronizado'].astype(str) + ' ' + df_2025['Descricao_padronizado'].astype(str)

# Calcular os embeddings
embeddings_24 = model.encode(texts_24)
print('embeddings_24')
embeddings_25 = model.encode(texts_25)
print('embeddings_25')

# Separar Projeto e Descrição para cálculo
project_embeddings_24 = model.encode(df_2024['Projeto_padronizado'].astype(str))
print('project_embeddings_24')
project_embeddings_25 = model.encode(df_2025['Projeto_padronizado'].astype(str))
print('project_embeddings_25')

description_embeddings_24 = model.encode(df_2024['Descricao_padronizado'].astype(str))
print('description_embeddings_24')
description_embeddings_25 = model.encode(df_2025['Descricao_padronizado'].astype(str))
print('description_embeddings_25')

embeddings_24
embeddings_25
project_embeddings_24
project_embeddings_25
description_embeddings_24
description_embeddings_25


In [ ]:
# Calcular similaridade de cosseno para Projeto e Descrição
similarity_matrix = cosine_similarity(embeddings_25, embeddings_24)
project_similarity_matrix = cosine_similarity(project_embeddings_25, project_embeddings_24)
description_similarity_matrix = cosine_similarity(description_embeddings_25, description_embeddings_24)

# Definir pesos para Projeto e Descrição
weight_project = 0.5
weight_description = 0.5

# Calcular similaridade ponderada combinada
combined_similarity_matrix = (
    weight_project * project_similarity_matrix +
    weight_description * description_similarity_matrix
)
combined_similarity_matrix_argmax_row = np.argmax(combined_similarity_matrix, axis=1)
combined_similarity_matrix_argmax_row.shape

(1224,)

In [ ]:
len(combined_similarity_matrix_argmax_row)

1224

### Solução: Mapear linhas que dão Match referenciando essas linhas em uma coluna.

In [ ]:
df_merged['similarity'] = 0
df_merged['relacionamento'] = '-'
df_merged['diff_valor'] = '-'
df_merged['perc_diff_valor'] = '-'
df_merged['valor_2024'] = '-'


In [ ]:
threshold_min = 0.9
limiar = df_2025.index.max()

for index_2025, index_2024 in enumerate(combined_similarity_matrix_argmax_row):
    index_with_limiar = index_2024 + limiar + 1

    # Marcar o relacionamento
    df_merged.loc[index_2025, 'relacionamento'] = index_with_limiar
    
    # Recuperar os valores correspondentes de ambas as tabelas
    valor_2024 = df_merged.loc[index_with_limiar, 'Valor']
    valor_2025 = df_merged.loc[index_2025, 'Valor']
    
    # Tentar converter para float e realizar os cálculos se possível
    try:
        valor_2024 = float(valor_2024)
        valor_2025 = float(valor_2025)
        
        # Calcular a diferença e a diferença percentual, evitando divisão por zero
        df_merged.loc[index_2025, 'diff_valor'] = valor_2025 - valor_2024
        df_merged.loc[index_2025, 'perc_diff_valor'] = ((valor_2025 - valor_2024) /  valor_2024) if valor_2024 != 0 else 'Não possível'
        df_merged.loc[index_2025, 'valor_2024'] = valor_2024
    
    except (ValueError, TypeError):
        # Caso os valores não sejam numéricos ou conversíveis para numéricos, definir como 'Não possível'
        df_merged.loc[index_2024, 'diff_valor'] = 'Não possível'
        df_merged.loc[index_2024, 'perc_diff_valor'] = 'Não possível'

    # Adicionar o valor de similaridade
    df_merged.loc[index_2025, 'similarity'] = combined_similarity_matrix[index_2025][index_2024]
    df_merged.loc[index_2025, 'Projeto_2024'] =  df_merged.loc[index_with_limiar, 'Projeto']
    df_merged.loc[index_2025, 'Descricao_2024'] = df_merged.loc[index_with_limiar, 'Descricao']


In [ ]:
df_merged = df_merged.reset_index()

In [ ]:
df_merged.to_excel('demandas_2025_match_2024.xlsx')

## Solução: Mapear e concatenar linhas do dataframe de 2024 e 2025

In [ ]:
df_2024 = df_merged[df_merged['Ano_Cotacao'] == 2024].reset_index(drop=True)
df_2025 = df_merged[df_merged['Ano_Cotacao'] == 2025].reset_index(drop=True)

In [ ]:
df_2024.columns = [col+'_2024' for col in df_2024.columns]
df_2025.columns = [col+'_2025' for col in df_2025.columns]

In [ ]:
df_2025['similarity'] = 0
df_2025['relacionamento'] = '-'
df_2025['diff_valor'] = '-'
df_2025['perc_diff_valor'] = '-'
df_2025['valor_2024'] = '-'

In [ ]:
df_2025.columns

Index(['Empresa_2025', 'Gestor_TI_2025', 'Codigo_2025', 'Projeto_2025',
       'Descricao_2025', 'Tema_2025', 'Valor_Ativacao_de_Pessoas_2025',
       'Benfeitorias_em_Propriedade_de_Terceiros_2025', 'Instalacoes_2025',
       'Maquinas_e_Equipamentos_Medicos_2025',
       'Maquinas_e_Equipamentos_Outros_2025', 'Moveis_e_Utensilios_2025',
       'Equipamentos_de_Informatica_2025', 'Licencas_de_Uso_de_Software_2025',
       'Software_Desenvolvido_Internamente_2025',
       'Produtos_Desenvolvidos_Internamente_2025', 'Valor_2025',
       'Centro_Custo_2025', 'Ano_Cotacao_2025', 'Projeto_padronizado_2025',
       'Descricao_padronizado_2025', 'similarity', 'relacionamento',
       'diff_valor', 'perc_diff_valor', 'valor_2024'],
      dtype='object')

In [ ]:
threshold_min = 0.9

for index_2025, index_2024 in enumerate(combined_similarity_matrix_argmax_row):
    if combined_similarity_matrix[index_2025][index_2024] < threshold_min:
        continue

    # Marcar o relacionamento
    df_2025.loc[index_2025, 'relacionamento'] = index_2024
    # Recuperar os valores correspondentes de ambas as tabelas
    valor_2024 = df_2024.loc[index_2024, 'Valor_2024']
    valor_2025 = df_2025.loc[index_2025, 'Valor_2025']
    
    
    # Tentar converter para float e realizar os cálculos se possível
    try:
        valor_2024 = float(valor_2024)
        valor_2025 = float(valor_2025)
        
        # Calcular a diferença e a diferença percentual, evitando divisão por zero
        df_2025.loc[index_2025, 'diff_valor'] = valor_2025 - valor_2024
        df_2025.loc[index_2025, 'perc_diff_valor'] = ((valor_2025 - valor_2024) /  valor_2024) if valor_2024 != 0 else 'Não possível'
        df_2025.loc[index_2025, 'valor_2024'] = valor_2024
    
    except (ValueError, TypeError):
        # Caso os valores não sejam numéricos ou conversíveis para numéricos, definir como 'Não possível'
        df_2025.loc[index_2024, 'diff_valor'] = 'Não possível'
        df_2025.loc[index_2024, 'perc_diff_valor'] = 'Não possível'

    # Adicionar o valor de similaridade
    df_2025.loc[index_2025, 'similarity'] = combined_similarity_matrix[index_2025][index_2024]
    #df_2025.loc[index_2025, 'Projeto_2024'] =  df_merged.loc[index_with_limiar, 'Projeto']
    #df_2025.loc[index_2025, 'Descricao_2024'] = df_merged.loc[index_with_limiar, 'Descricao']

    df_2025.loc[index_2025, df_2024.columns] = df_2024.loc[index_2024, df_2024.columns]

#### Remove linhas que já foram encontradas em 2024

In [ ]:
df_2024 = df_2024[~df_2024.index.isin([row for row in df_2025['relacionamento'].unique() if row != '-'])]
df_2024.shape

(298, 21)

In [ ]:
cols_2024 = df_2024.columns
cols_2025 = df_2025.columns

df_2024[list(set(cols_2025) - set(cols_2024))] = '-'

df_2024 = df_2024[cols_2025]
df_2024.shape

(298, 47)

In [ ]:
df_2025_2024 = pd.concat([df_2025, df_2024])
df_2025_2024.shape

(1522, 47)

In [ ]:
df_2025_2024.to_excel('demandas_2025_match_2024.xlsx')

### END

In [ ]:
# Definir um limiar para considerar textos como similares
threshold_min = 0.95
threshold_max = 0.99

# Encontrar pares de textos similares
similar_texts = np.argwhere(((combined_similarity_matrix > threshold_min) & (combined_similarity_matrix <= threshold_max)))
# Remover comparações de auto-similaridade e duplicatas
similar_texts = [v for v in similar_texts if v[0] != v[1]]

# Exibir os pares de textos similares e suas similaridades
print("Pares de textos similares:")
for i, j in similar_texts:
    print(f"Texto 1: {texts[i]}")
    print(f"Texto 2: {texts[j]}")
    print(f"Similaridade: {combined_similarity_matrix[i, j]:.4f}")
    print()

Pares de textos similares:
Texto 1: [Auditoria] LR ACL PARA WINDOWS - Pardini Licenças voltadas a aplicação do sistema de auditoria
Texto 2: [Auditoria] LR ACL PARA WINDOWS Licenças voltadas a aplicação do sistema de auditoria
Similaridade: 0.9867

Texto 1: [Auditoria] LR ACL PARA WINDOWS - Pardini Licenças voltadas a aplicação do sistema de auditoria
Texto 2: [Auditoria] LR ACL PARA WINDOWS Licenças voltadas a aplicação do sistema de auditoria
Similaridade: 0.9867

Texto 1: Google Analytics Premium - Pardini Ferramenta que mensura a audiência de todos os sites do Grupo, trazendo indicadores para melhora de performance, criação de novos produtos, ajustes de jornadas, cruzamento de dados offline com os dados de navegação dos sites para melhorias, correções e novas jornadas on e offline.
Texto 2: [MKT]Google Analytics Premium Pardini Ferramenta que mensura a audiência de todos os sites do Grupo, trazendo indicadores para melhora de performance, criação de novos produtos, ajustes de jorna

In [ ]:
# # Gerar embeddings para cada texto
# embeddings = model.encode(texts)

# print(embeddings)

[[-0.37075964 -0.14254816 -0.21975896 ... -0.21431781  0.21226157
  -0.00484152]
 [ 0.29147866  0.02444368 -0.19543156 ... -0.00129475 -0.11732929
  -0.10194118]
 [-0.26879755  0.10598219 -0.12942375 ...  0.04164211  0.13630436
   0.16277006]
 ...
 [ 0.05210876 -0.0295249   0.06104898 ... -0.02256301  0.06818666
   0.09140446]
 [ 0.05210876 -0.0295249   0.06104898 ... -0.02256301  0.06818666
   0.09140446]
 [ 0.05210872 -0.02952482  0.061049   ... -0.02256286  0.06818669
   0.09140431]]


In [ ]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# # Calcular a similaridade de cosseno entre todos os pares de frases
# similarity_matrix = cosine_similarity(embeddings)

# # Definir um limiar para considerar textos como similares
# threshold_min = 0.93
# threshold_max = 1.1

# # Encontrar pares de textos similares
# similar_texts = np.argwhere(((similarity_matrix > threshold_min) & (similarity_matrix <= threshold_max)))
# similar_texts = [v for v in similar_texts if v[0] != v[1]]
# # Exibir os pares de textos similares e suas similaridades
# print("Pares de textos similares:")
# for i, j in similar_texts:
#     if i == j:
#         continue
#     if i < j:  # Evitar duplicação e comparações de auto-similaridade
#         print(f"Texto 1: {texts[i]}")
#         print(f"Texto 2: {texts[j]}")
#         print(f"Similaridade: {similarity_matrix[i, j]:.4f}")
#         print()

Pares de textos similares:
Texto 1: [Marketing] LR Dinamize Ferramenta de CRM (Disparo de e-mails marketing) para todas as marcas do Grupo Fleury
Texto 2: [Marketing] LR Dinamize - Pardini Ferramenta de CRM (Disparo de e-mails marketing) para todas as marcas do Grupo Fleury
Similaridade: 0.9834

Texto 1: [Marketing] LR Dinamize Ferramenta de CRM (Disparo de e-mails marketing) para todas as marcas do Grupo Fleury
Texto 2: [Marketing] LR Dinamize - Novos Elos Ferramenta de CRM (Disparo de e-mails marketing) para todas as marcas do Grupo Fleury
Similaridade: 0.9797

Texto 1: [Marketing] LR Dinamize - Pardini Ferramenta de CRM (Disparo de e-mails marketing) para todas as marcas do Grupo Fleury
Texto 2: [Marketing] LR Dinamize - Novos Elos Ferramenta de CRM (Disparo de e-mails marketing) para todas as marcas do Grupo Fleury
Similaridade: 0.9829

Texto 1: [Marketing] RankMyApp - Pardini Ferramenta de análise de ASO para aplicativos do Grupo. Utilizada para definição de estratégias e implemen

In [ ]:
len(similar_texts)

2518

In [ ]:
# # Usar um conjunto para rastrear combinações únicas
# unique_combinations = set()

# # Lista para armazenar as combinações únicas
# filtered_texts = []

# for pair in similar_texts:
#     # Ordenar a tupla para garantir que [1, 2] e [2, 1] sejam iguais
#     normalized_pair = tuple(sorted(pair))
    
#     # Verificar se a combinação normalizada já está no conjunto
#     if normalized_pair not in unique_combinations:
#         unique_combinations.add(normalized_pair)
#         filtered_texts.append(pair)

# filtered_texts

[array([0, 0]),
 array([1, 1]),
 array([2, 2]),
 array([3, 3]),
 array([4, 4]),
 array([5, 5]),
 array([6, 6]),
 array([7, 7]),
 array([8, 8]),
 array([9, 9]),
 array([10, 10]),
 array([11, 11]),
 array([12, 12]),
 array([13, 13]),
 array([14, 14]),
 array([15, 15]),
 array([16, 16]),
 array([17, 17]),
 array([18, 18]),
 array([19, 19]),
 array([20, 20]),
 array([21, 21]),
 array([22, 22]),
 array([23, 23]),
 array([24, 24]),
 array([25, 25]),
 array([26, 26]),
 array([27, 27]),
 array([28, 28]),
 array([29, 29]),
 array([30, 30]),
 array([31, 31]),
 array([32, 32]),
 array([33, 33]),
 array([34, 34]),
 array([35, 35]),
 array([36, 36]),
 array([37, 37]),
 array([38, 38]),
 array([39, 39]),
 array([40, 40]),
 array([41, 41]),
 array([42, 42]),
 array([43, 43]),
 array([44, 44]),
 array([45, 45]),
 array([46, 46]),
 array([47, 47]),
 array([48, 48]),
 array([49, 49]),
 array([49, 50]),
 array([49, 51]),
 array([50, 50]),
 array([50, 51]),
 array([51, 51]),
 array([52, 52]),
 array([53, 

In [ ]:
# import networkx as nx


# # Criar um grafo
# G = nx.Graph()

# # Adicionar arestas ao grafo
# G.add_edges_from(similar_texts)

# # Encontrar todos os componentes conectados
# connected_components = list(nx.connected_components(G))

# # Converter os componentes conectados para uma lista de listas
# grouped_values = [list(component) for component in connected_components]

# print(grouped_values)


[[49, 50], [56, 55], [57, 58], [168, 167], [206, 207], [232, 231], [273, 236], [248, 247], [249, 250], [252, 253], [256, 255], [257, 266], [281, 282], [283, 284], [286, 287], [288, 289], [292, 293, 294], [512, 513], [530, 534], [546, 550], [576, 566], [577, 567], [569, 570], [578, 579], [584, 583], [585, 586], [590, 591], [593, 594], [595, 596], [597, 598], [600, 601], [603, 604], [608, 609], [610, 611], [614, 615], [616, 617], [619, 620], [635, 621], [622, 623], [628, 629], [630, 631], [640, 641], [642, 643], [646, 647], [648, 649], [652, 653], [654, 655], [657, 658], [669, 670], [717, 718], [760, 761, 762, 763, 764, 765, 766], [774, 775], [817, 821], [825, 818, 826, 822], [824, 819], [832, 834, 830], [848, 849], [852, 871], [872, 853], [881, 882], [946, 948, 950, 951], [960, 961, 962, 963, 964, 965, 954, 955, 956, 957, 958, 959], [966, 967, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977], [978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989], [992, 993, 994, 995, 996, 997, 9

In [ ]:
import networkx as nx

# Criar um grafo
G = nx.Graph()

# Adicionar arestas ao grafo com pesos
for (i, j) in similar_texts:
    # Definir o peso usando a matriz de similaridade
    weight = similarity_matrix[i, j]
    G.add_edge(i, j, weight=weight)

# Encontrar todos os componentes conectados
connected_components = list(nx.connected_components(G))

# Converter os componentes conectados para uma lista de listas
grouped_values = [list(component) for component in connected_components]

print("Grupos de nós conectados:")
print(grouped_values)

Grupos de nós conectados:
[[8, 521, 118], [205, 13], [17, 447], [212, 20], [22, 223], [224, 23], [26, 931], [28, 933], [30, 935], [936, 932, 31], [937, 34], [264, 36], [298, 38], [302, 39], [40, 301], [45, 303], [306, 47], [48, 311], [49, 316], [50, 318], [51, 319], [320, 52], [322, 53], [323, 54], [325, 55], [56, 327], [328, 57], [329, 58], [331, 63], [333, 71], [75, 334], [76, 335], [336, 77], [337, 78], [338, 79], [80, 339], [81, 340], [82, 341], [83, 342], [84, 343], [344, 85], [345, 86], [115, 460], [139, 510], [148, 461], [160, 1057], [161, 1058], [168, 1060], [1064, 169], [178, 419], [179, 420], [421, 182], [422, 183], [184, 423], [425, 186], [426, 188], [193, 1295], [1296, 194], [448, 209], [214, 1271], [216, 217], [400, 230], [942, 237, 238], [914, 245], [249, 917], [250, 253, 918], [251, 919], [920, 252], [258, 924], [259, 926], [260, 927], [848, 363], [850, 364], [891, 365], [866, 366], [867, 367], [368, 868], [369, 869], [370, 870], [371, 871], [872, 372], [873, 373], [874,

In [ ]:
print("\nPesos das arestas:")
for u, v, data in G.edges(data=True):
    print(f"Aresta ({u}, {v}) com peso {data['weight']:.2f}")


Pesos das arestas:
Aresta (8, 118) com peso 0.98
Aresta (8, 521) com peso 0.98
Aresta (118, 521) com peso 1.00
Aresta (13, 205) com peso 1.00
Aresta (17, 447) com peso 0.90
Aresta (20, 212) com peso 1.00
Aresta (22, 223) com peso 1.00
Aresta (23, 224) com peso 1.00
Aresta (26, 931) com peso 1.00
Aresta (28, 933) com peso 1.00
Aresta (30, 935) com peso 1.00
Aresta (31, 932) com peso 0.95
Aresta (31, 936) com peso 1.00
Aresta (932, 936) com peso 0.95
Aresta (34, 937) com peso 1.00
Aresta (36, 264) com peso 1.00
Aresta (38, 298) com peso 1.00
Aresta (39, 302) com peso 0.94
Aresta (40, 301) com peso 0.96
Aresta (45, 303) com peso 1.00
Aresta (47, 306) com peso 1.00
Aresta (48, 311) com peso 1.00
Aresta (49, 316) com peso 1.00
Aresta (50, 318) com peso 1.00
Aresta (51, 319) com peso 1.00
Aresta (52, 320) com peso 1.00
Aresta (53, 322) com peso 1.00
Aresta (54, 323) com peso 1.00
Aresta (55, 325) com peso 1.00
Aresta (56, 327) com peso 1.00
Aresta (57, 328) com peso 1.00
Aresta (58, 329) co

## Definir os grupos

In [ ]:
import numpy as np 

# Adicionar uma coluna de grupo inicializada como NaN
df['group'] = np.nan

# Adicionar uma coluna para a média dos valores das conexões
df['mean_similarity'] = np.nan

# Mapear cada nó para o seu grupo
node_to_group = {}
for idx, group in enumerate(grouped_values):
    for node in group:
        node_to_group[node] = idx

# Preencher a coluna 'group' no DataFrame
df['group'] = df.index.map(node_to_group)

print(df['group'].value_counts())

150.0    12
155.0    12
147.0    12
151.0    12
152.0    12
         ..
70.0      2
69.0      2
68.0      2
67.0      2
83.0      2
Name: group, Length: 166, dtype: int64


## Define a força do grupo

In [ ]:
# Calcular a média dos valores de similaridade para cada grupo
group_similarities = {}
for group in grouped_values:
    # Encontrar todas as arestas dentro do grupo
    edges = [(u, v) for u in group for v in group if u < v]
    if edges:
        # Calcular a soma das similaridades para as arestas do grupo
        total_similarity = sum(G[u][v]['weight'] for u, v in edges if G.has_edge(u, v))
        avg_similarity = total_similarity / len(edges)
    else:
        avg_similarity = 0
    group_aux = int(df.iloc[group[0]]['group'])
    group_similarities[group_aux] = avg_similarity  # Usar o primeiro nó como identificador de grupo

# Preencher a coluna 'mean_similarity' no DataFrame com a média de similaridade para cada grupo
df['mean_similarity'] = df['group'].map(group_similarities)

In [ ]:
df['mean_similarity'].describe()

count    523.000000
mean       0.992559
std        0.020657
min        0.824479
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000001
Name: mean_similarity, dtype: float64

In [ ]:
df[df['mean_similarity'] < .8]

,Projeto,Descricao,group,mean_similarity


In [ ]:
df['group'].describe()

count    523.000000
mean     106.219885
std       50.675924
min        0.000000
25%       63.500000
50%      128.000000
75%      149.000000
max      165.000000
Name: group, dtype: float64

In [ ]:
df['mean_similarity'] = df['mean_similarity'].round(6)

In [ ]:
df.to_excel('de_para_projetos.xlsx')

In [ ]:
G[u][v]['weight']